### 1.1
Сгенерировать выборки X и Y объемом 500 величин, подчиняющихся распределениям Гаусса с параметрами N(3,15) и N(3,30) соответственно
Проверить критерием знаков гипотезу H0{"X - Y = 0"}

In [878]:
import numpy as np
import scipy.stats as sps


In [879]:

def build_gauss(mean, var, size):
    return sps.norm(mean,np.sqrt(var)).rvs(size)


def sign_test(x, y):

    plus = np.sum(x > y) 
    minus = np.sum(x < y)  

    n = plus + minus   
    w = min(plus, minus) 
    stats = (w - n/2) / np.sqrt(n/4)
    
    pvalue = sps.binomtest(plus, n).pvalue
    stats_scipy = sps.binomtest(plus, n).statistic

    if pvalue < 0.05:
        print(f'Decline H0, pvalue = {pvalue}, stats = {stats}, stats_scipy = {stats_scipy}')

    else:
        print(f'Accept H0, pvalue = {pvalue}, stats = {stats}, stats_scipy = {stats_scipy}')



In [880]:
#np.random.seed(333)
X = build_gauss(3,15,500)
Y = build_gauss(3,30,500)

print('1.1: N(3,15), N(3,30):')
sign_test(X,Y)

1.1: N(3,15), N(3,30):
Accept H0, pvalue = 1.0, stats = 0.0, stats_scipy = 0.5


### 1.2
Сгенерировать выборки X и Y объемом 500 величин, подчиняющихся распределениям Гаусса с параметрами N(3,15) и N(3.3,30) соответственно

In [881]:
X = build_gauss(3,15,500)
Y_new = build_gauss(-1 ,30,500)

print('1.2 N(3,15), N(3.3,30):')
sign_test(X,Y_new)

1.2 N(3,15), N(3.3,30):
Decline H0, pvalue = 4.2843741010351617e-29, stats = -11.001454449298965, stats_scipy = 0.746


### 2.1, 2.2 
Аналогично с ранговыми критериями (4 ранга)

In [882]:
def wilcoxon_test_scipy(x, y):
    dif = x - y
    dif = dif[dif!=0]
    stats, pvalue = sps.wilcoxon(dif)

    if pvalue < 0.05:
        print(f'Decline H0, pvalue = {pvalue}, stats_scipy = {stats}')
    else:
        print(f'Accept H0, pvalue = {pvalue}, stats_scipy = {stats}')

In [883]:
print('2.1 N(3,15), N(3,30)')
wilcoxon_test_scipy(X,Y)

print('\n2.1 N(3,15), N(3.3,30)')
wilcoxon_test_scipy(X,Y_new)

2.1 N(3,15), N(3,30)
Accept H0, pvalue = 0.9600278040882478, stats_scipy = 62463.0

2.1 N(3,15), N(3.3,30)
Decline H0, pvalue = 5.214105045074739e-34, stats_scipy = 23327.0


In [884]:

def wilcoxon_test(x, y):

    d = x - y
    signs = np.sign(d)
    d = abs(d[d!=0])
    indxs = np.argsort(d)
    d = d[indxs]
    signs = signs[indxs]
   
    n = len(d)
    ranks2 = sps.rankdata(d)
    indxs = np.argsort(sps.rankdata(d))
    signs = signs[indxs]
    #print(signs)
    
    #print(ranks2)
    stats = np.sum(ranks2 * signs) / np.sqrt(((n + 1) * (2*n + 1) * n)/6)

    l = sps.norm.ppf(0.025)
    r = sps.norm.ppf(0.0975)
    pvalue = 2 * (1 - sps.norm.cdf(np.abs(stats)))
    
    if pvalue < 0.05 or l > stats  or stats > -l:
        print(f'Decline H0, pvalue = {pvalue}, stats = {stats}')
    else:
        print(f'Accept H0, pvalue = {pvalue}, stats = {stats}')



In [885]:
print('2.1 N(3,15), N(3,30)')
wilcoxon_test(X,Y)

print('\n2.1 N(3,15), N(3.3,30)')
wilcoxon_test(X,Y_new)

2.1 N(3,15), N(3,30)
Accept H0, pvalue = 0.960027804088248, stats = -0.050118692383726286

2.1 N(3,15), N(3.3,30)
Decline H0, pvalue = 0.0, stats = 12.157804773430097


### 3.1 
Сгенерировать выборку X объемом 600 величин, подчиняющихся распределениям Гаусса с параметрами 5, 7 (N(5; 7))
### 3.2
Взять тот же X и сделать Y = 2X - 5 + eps, eps ~ N(0,1)
Повторить то же самое + сравнить Кендела и Пирсона
### 3.3
X те же, Y = 0.01 * X + eps, eps ~ N(0,1)
Повторить то же самое + сравнить Кендела и Пирсона

In [886]:
def candal(x,y):

    n = len(x)
    coef_sum = coef_sum = [(np.sign(x[i] - x[j]) * (np.sign(y[i] - y[j]))) 
                           for i in range(n) for j in range(i+1, n)]
    tot_sum = np.sum(coef_sum)

    r = (2 / (n * (n - 1))) * tot_sum
    var = np.sqrt((2 * (2 * n + 5)) / (9 * n * (n - 1)))

    stats = r / var
    q = sps.norm.ppf(0.025)
    pvalue_candal = 2 * (1 - sps.norm.cdf(np.abs(stats)))

    pearson = np.corrcoef(x,y)[0,1]
    stats_pearson = np.sqrt(n - 2) * pearson/np.sqrt(1 - pearson ** 2)
    pvalue_pearson = 2 * (1 - sps.t.cdf(abs(stats_pearson), df=n-2))
    q_t = sps.t.ppf(0.025, df=n-2)

    if  pvalue_candal < 0.05 or q > stats  or stats < q or stats > -q:
        print(f'Candal: Decline H0, pvalue = {pvalue_candal}, stats = {stats}')

    else:
        print(f'Candal: Accept H0, pvalue = {pvalue_candal}, stats = {stats}')

    if  pvalue_pearson < 0.05 or q_t > stats  or stats_pearson < q_t or stats_pearson > -q_t:
        print(f'Pearson: Decline H0, pvalue = {pvalue_pearson}, stats = {stats_pearson}')

    else:
        print(f'Pearson: Accept H0, pvalue = {pvalue_pearson}, stats = {stats_pearson}')




In [887]:
X = build_gauss(5,7,600)

x1, x2 = np.array_split(X,2)
print('3.1 N(5,7) 600 элементов, на две части:\n')
candal(x1,x2)

Y = 0.001 * X - 5 + build_gauss(0, 1, 600)
print('\n3.2 N(5,7) Y = 2*X - 5 + eps, на две части:\n')
candal(X,Y)

3.1 N(5,7) 600 элементов, на две части:



Candal: Accept H0, pvalue = 0.18150516383005044, stats = -1.3361349634646227
Pearson: Accept H0, pvalue = 0.14991276437517054, stats = -1.4435615309405148

3.2 N(5,7) Y = 2*X - 5 + eps, на две части:

Candal: Accept H0, pvalue = 0.2011556558515868, stats = -1.2782659884112393
Pearson: Accept H0, pvalue = 0.1465844056439094, stats = -1.4535906738897


### 4.1 
Сгенерировать выборку X объемом 500 величин, подчиняющихся распределениям Гаусса с параметрами 3, 4 (N(3; 4))
Проверить критерий отсутствия автокорреляции
### 4.2
Взять тот же Х, первый элемент выборки оставить тем же, остальные X_j = X_j - 2 * X_j-1 + eps, eps ~ N(0,1)
Проверить критерий отсутствия автокорреляции

In [888]:
def check_autocorr(x):
    n = len(x)
    coef_sum = np.sum([x[i]*x[i+1] for i in range(n - 1)])

    t = n * coef_sum - (np.sum(x) ** 2) + n * x[0] * x[n - 1]
    t /= n * np.sum(x ** 2) - (np.sum(x)) ** 2

    stats = (t + 1 / (n - 1)) / np.sqrt((n * (n - 3)) / ((n + 1) * ((n - 2) ** 2)))

    q = sps.norm.ppf(0.025)

    pvalue = 2 * (1 - sps.norm.cdf(np.abs(stats)))

    if pvalue < 0.05 or q > stats  or stats > -q:
        print(f'Decline H0, pvalue = {pvalue}, stats = {stats}')
        
    else:
        print(f'Accept H0, pvalue = {pvalue}, stats = {stats}') 

In [889]:
X = build_gauss(3, 4, 500)

print('\n4.1 N(3,4):')
check_autocorr(X)

eps = build_gauss(0,1,500)

X[1:] = [X[i] - 2 * X[i-1] + eps[i] for i in range(1, len(X))]

print('\n4.2 N(3,4) первый элемент выборки оставили тем же, остальные X_j = X_j - 2 * X_j-1 + eps, eps ~ N(0,1):')
check_autocorr(X)


4.1 N(3,4):
Accept H0, pvalue = 0.9941081846144588, stats = -0.007384362626689111

4.2 N(3,4) первый элемент выборки оставили тем же, остальные X_j = X_j - 2 * X_j-1 + eps, eps ~ N(0,1):
Decline H0, pvalue = 2.220446049250313e-16, stats = -8.27460112578239
